# Agentic Core Module Example

This notebook demonstrates how to use the agentic core module to create and work with an AI agent that can:
- Process messages
- Use tools
- Maintain memory of conversations
- Plan and execute tasks

The agent core provides a flexible structure for building agentic AI applications based on modular components.

## Setup and Imports

First, we'll import the necessary modules from the agentic core package.

In [9]:
import sys
import os
import logging

# Add the parent directory to sys.path to allow importing the agentic module
# This is only needed if the agentic module is not installed or in the Python path
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..', '..', '..')))

# Import the core components
from agentic.core import AgentCore, AgentConfig
from agentic.core.tools import Tool, ToolResult

# Configure logging
logging.basicConfig(level=logging.INFO, 
                   format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

## Creating a Custom Tool

Let's create a simple tool that the agent can use. This calculator tool can perform basic arithmetic operations.

In [10]:
class CalculatorTool(Tool):
    """A simple calculator tool for demonstration purposes."""
    
    def __init__(self):
        """Initialize the calculator tool."""
        super().__init__(
            name="calculator",
            description="Performs basic arithmetic operations (add, subtract, multiply, divide)"
        )
    
    def execute(self, operation: str, a: float, b: float) -> ToolResult:
        """Execute the calculator operation.
        
        Args:
            operation: The operation to perform (add, subtract, multiply, divide)
            a: First number
            b: Second number
            
        Returns:
            Result of the calculation
        """
        try:
            if operation == "add":
                result = a + b
            elif operation == "subtract":
                result = a - b
            elif operation == "multiply":
                result = a * b
            elif operation == "divide":
                if b == 0:
                    return ToolResult.error_result("Division by zero is not allowed")
                result = a / b
            else:
                return ToolResult.error_result(f"Unknown operation: {operation}")
            
            return ToolResult.success_result(result)
        except Exception as e:
            return ToolResult.error_result(f"Error in calculator: {str(e)}")
    
    def get_schema(self) -> dict:
        """Get the schema for the calculator tool."""
        return {
            "parameters": {
                "operation": {
                    "type": "string",
                    "description": "The operation to perform (add, subtract, multiply, divide)",
                    "required": True
                },
                "a": {
                    "type": "number",
                    "description": "First number",
                    "required": True
                },
                "b": {
                    "type": "number",
                    "description": "Second number",
                    "required": True
                }
            },
            "returns": {
                "type": "number",
                "description": "Result of the calculation"
            }
        }

# Let's test our calculator tool directly
calculator = CalculatorTool()
result = calculator.execute(operation="add", a=5, b=3)
print(f"Calculator result: {result.data}")

Calculator result: 8


## Creating the Agent

Now let's create an agent with a custom configuration and register our calculator tool.

In [11]:
# Create agent configuration
config = AgentConfig(
    verbose=True,
    planning_enabled=True,
    long_term_memory_enabled=True,
    short_term_memory_capacity=15  # Store more messages than the default
)

# Create agent core
agent = AgentCore(config)

# Register our calculator tool
agent.register_tool(calculator)

# Get all registered tools
registered_tools = agent.tool_registry.get_all_tools()
print(f"Registered tools: {[tool.name for tool in registered_tools]}")

2025-03-28 15:22:51,692 - agentic.core - INFO - Registering tool: calculator
2025-03-28 15:22:51,692 - agentic.core - INFO - Registering tool: calculator


Registered tools: ['calculator']


## Interacting with the Agent

Let's interact with the agent and see how it responds to various messages.

In [12]:
# Send a simple greeting message
response = agent.process_message("Hello, who are you?")
print(f"User: Hello, who are you?")
print(f"Agent: {response}\n")

# Ask about the agent's capabilities
response = agent.process_message("What can you do?")
print(f"User: What can you do?")
print(f"Agent: {response}\n")

# Ask if the agent can perform a task
task = "Can you calculate 42 divided by 6?"
assessment = agent.can_accomplish(task)
print(f"User: {task}")
print(f"Task assessment: {assessment['can_accomplish']}")
print(f"Reason: {assessment['reason']}")

# Ask the agent to perform a task
response = agent.process_message(task)
print(f"Agent: {response}\n")

2025-03-28 15:22:51,706 - agentic.core - INFO - Processing message from user: Hello, who are you?
2025-03-28 15:22:51,706 - agentic.core - INFO - Processing message from user: Hello, who are you?
2025-03-28 15:22:51,708 - agentic.planner - INFO - Generating plan for goal: Hello, who are you?
2025-03-28 15:22:51,708 - agentic.planner - INFO - Generating plan for goal: Hello, who are you?
2025-03-28 15:22:51,709 - agentic.planner - INFO - Generated plan with 3 steps
2025-03-28 15:22:51,709 - agentic.planner - INFO - Generated plan with 3 steps
2025-03-28 15:22:51,710 - agentic.core - INFO - Executing plan for message: Hello, who are you?
2025-03-28 15:22:51,710 - agentic.core - INFO - Executing plan for message: Hello, who are you?
2025-03-28 15:22:51,711 - agentic.executor - INFO - Executing step: Analyze the request to understand the user's goal
2025-03-28 15:22:51,711 - agentic.executor - INFO - Executing step: Analyze the request to understand the user's goal
2025-03-28 15:22:51,712 

User: Hello, who are you?
Agent: I understand your request. However, I currently don't have the necessary tools to accomplish this task. As my capabilities develop, I'll be able to assist with more complex requests.

User: What can you do?
Agent: I understand your request. However, I currently don't have the necessary tools to accomplish this task. As my capabilities develop, I'll be able to assist with more complex requests.

User: Can you calculate 42 divided by 6?
Task assessment: True
Reason: The task can be accomplished with the current tools.
Agent: I understand your request. However, I currently don't have the necessary tools to accomplish this task. As my capabilities develop, I'll be able to assist with more complex requests.



## Examining Memory

Let's examine the agent's memory to see what it has stored from our conversation.

In [13]:
# Get conversation history
conversation_history = agent.memory.get_conversation_history(as_text=True)
print("Conversation History:")
print(conversation_history)

# Get recent messages
recent_messages = agent.memory.get_recent_messages(limit=3)
print("\nMost Recent 3 Messages:")
for msg in recent_messages:
    print(f"{msg.sender.capitalize()}: {msg.content}")

Conversation History:
Agent: I understand your request. However, I currently don't have the necessary tools to accomplish this task. As my capabilities develop, I'll be able to assist with more complex requests.
User: Can you calculate 42 divided by 6?
Agent: I understand your request. However, I currently don't have the necessary tools to accomplish this task. As my capabilities develop, I'll be able to assist with more complex requests.

Most Recent 3 Messages:
Agent: I understand your request. However, I currently don't have the necessary tools to accomplish this task. As my capabilities develop, I'll be able to assist with more complex requests.


## Creating a New Tool Dynamically

Let's demonstrate how to create and register a new tool dynamically during runtime.

In [14]:
# Create a simple weather tool
class WeatherTool(Tool):
    """A simple weather tool for demonstration purposes."""
    
    def __init__(self):
        """Initialize the weather tool."""
        super().__init__(
            name="weather",
            description="Gets the weather for a given location (simulated)"
        )
    
    def execute(self, location: str) -> ToolResult:
        """Get the weather for a location (simulated).
        
        Args:
            location: The location to get weather for
            
        Returns:
            Simulated weather information
        """
        # This is a simulated response
        import random
        weather_conditions = ["sunny", "cloudy", "rainy", "snowy", "windy"]
        temperature = random.randint(0, 35)  # Celsius
        condition = random.choice(weather_conditions)
        
        return ToolResult.success_result({
            "location": location,
            "temperature": temperature,
            "condition": condition,
            "humidity": random.randint(30, 90),
            "timestamp": "2025-03-28T12:00:00Z"  # Current date in the context
        })
    
    def get_schema(self) -> dict:
        """Get the schema for the weather tool."""
        return {
            "parameters": {
                "location": {
                    "type": "string",
                    "description": "The location to get weather for",
                    "required": True
                }
            },
            "returns": {
                "type": "object",
                "description": "Weather information"
            }
        }

# Register the new tool with the agent
weather_tool = WeatherTool()
agent.register_tool(weather_tool)

# Now let's test the new tool directly
weather_result = weather_tool.execute(location="Seattle")
print(f"Weather in Seattle: {weather_result.data}")

# Get updated list of tools
updated_tools = agent.tool_registry.get_all_tools()
print(f"Updated tool list: {[tool.name for tool in updated_tools]}")

2025-03-28 15:22:51,767 - agentic.core - INFO - Registering tool: weather
2025-03-28 15:22:51,767 - agentic.core - INFO - Registering tool: weather


Weather in Seattle: {'location': 'Seattle', 'temperature': 6, 'condition': 'snowy', 'humidity': 65, 'timestamp': '2025-03-28T12:00:00Z'}
Updated tool list: ['calculator', 'weather']


## Testing the Agent with New Capabilities

Let's see if the agent can now handle weather-related queries.

In [15]:
# Check if the agent can accomplish a weather task
weather_task = "What's the weather like in New York?"
assessment = agent.can_accomplish(weather_task)
print(f"User: {weather_task}")
print(f"Task assessment: {assessment['can_accomplish']}")
print(f"Reason: {assessment['reason']}")

# Process a weather request
response = agent.process_message(weather_task)
print(f"Agent: {response}\n")

2025-03-28 15:22:51,788 - agentic.core - INFO - Evaluating if task can be accomplished: What's the weather like in New York?
2025-03-28 15:22:51,788 - agentic.core - INFO - Evaluating if task can be accomplished: What's the weather like in New York?
2025-03-28 15:22:51,790 - agentic.planner - INFO - Generating plan for goal: What's the weather like in New York?
2025-03-28 15:22:51,790 - agentic.planner - INFO - Generating plan for goal: What's the weather like in New York?
2025-03-28 15:22:51,791 - agentic.planner - INFO - Generated plan with 3 steps
2025-03-28 15:22:51,791 - agentic.planner - INFO - Generated plan with 3 steps
2025-03-28 15:22:51,793 - agentic.core - INFO - Processing message from user: What's the weather like in New York?
2025-03-28 15:22:51,793 - agentic.core - INFO - Processing message from user: What's the weather like in New York?
2025-03-28 15:22:51,794 - agentic.planner - INFO - Generating plan for goal: What's the weather like in New York?
2025-03-28 15:22:51,

User: What's the weather like in New York?
Task assessment: True
Reason: The task can be accomplished with the current tools.
Agent: I understand your request. However, I currently don't have the necessary tools to accomplish this task. As my capabilities develop, I'll be able to assist with more complex requests.



## Planning and Execution

The agent uses a planning system to break down tasks into steps. Let's examine the current plan after our interactions.

In [16]:
# Access the current plan
current_plan = agent.planning_engine.current_plan

if current_plan:
    print(f"Current Plan Goal: {current_plan.goal}")
    print(f"Plan Status: {current_plan.status.value}")
    print("\nSteps:")
    
    for i, step in enumerate(current_plan.steps, 1):
        print(f"{i}. [{step.status.value}] {step.description}")
        if step.tool_name:
            print(f"   Tool: {step.tool_name}")
            print(f"   Args: {step.tool_args}")
        if step.result:
            print(f"   Result: {step.result}")
        if step.error:
            print(f"   Error: {step.error}")
else:
    print("No current plan available.")

Current Plan Goal: What's the weather like in New York?
Plan Status: completed

Steps:
1. [completed] Analyze the request to understand the user's goal
2. [completed] Determine if the goal can be accomplished with available tools
3. [completed] Inform the user about the capabilities that would be needed to accomplish the task


## Next Steps for Development

In a full implementation, the agent would need:

1. **Integration with a real LLM API** - The current implementation uses placeholders for LLM calls
2. **More tools** - Additional tools to expand the agent's capabilities
3. **Better planning logic** - Enhanced ability to create and execute complex plans
4. **Improved memory** - Vectorized or semantic search for more effective memory recall
5. **Self-improvement** - Ability for the agent to write and integrate its own tools

The modular design makes it easy to enhance these components independently.

## Conclusion

This notebook demonstrated the basic usage of the agentic core module. The framework provides a structured approach to building agentic AI applications with modular components for:

- Memory management
- Planning and execution
- Tool discovery and utilization
- Message processing

With these components in place, you can build sophisticated AI agents that can accomplish complex tasks using a variety of tools and capabilities.